In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
sqlite3 my_database.db < database.sql
https://github.com/ds4v/quan-ly-hoc-sinh/blob/main/DATABASE/Tables.sql
https://blog.streamlit.io/chat2vis-ai-driven-visualisations-with-streamlit-and-natural-language/
https://yia333.medium.com/building-a-real-time-data-visualization-solution-with-generative-ai-4a560f603696
https://github.com/vanna-ai/vanna
https://openscience.vn/chi-tiet-du-lieu/bo-du-lieu-ket-qua-hoc-tap-cua-hoc-sinh-11133
https://blog.langchain.dev/data-viz-agent/
https://github.com/vanna-ai/vanna-streamlit/blob/main/app.py

Để tạo một file `.db` từ một file `.sql`, bạn cần sử dụng một hệ quản trị cơ sở dữ liệu (DBMS) như SQLite, MySQL, PostgreSQL, hoặc tương tự. Dưới đây là cách làm với SQLite (một DBMS nhẹ và dễ sử dụng) để chuyển đổi file `.sql` thành file `.db`.

### Các bước thực hiện:

1. **Cài đặt SQLite**:
   - Nếu bạn chưa cài đặt SQLite, bạn có thể tải nó từ [trang web của SQLite](https://www.sqlite.org/download.html) và cài đặt trên hệ điều hành của bạn.

2. **Tạo file .db từ file .sql**:
   - Mở Terminal (hoặc Command Prompt trên Windows).
   - Chạy lệnh sau để tạo database mới từ file `.sql` của bạn:
   
     ```bash
     sqlite3 ten_file_moi.db < ten_file.sql
     ```
   
     Trong đó:
     - `ten_file_moi.db`: tên file database mới mà bạn muốn tạo (file `.db`).
     - `ten_file.sql`: tên của file SQL chứa các câu lệnh tạo cơ sở dữ liệu.

3. **Kiểm tra**:
   Sau khi thực thi lệnh trên, bạn sẽ có file `ten_file_moi.db` ở cùng thư mục với file `.sql`. Bạn có thể kiểm tra nội dung của database bằng cách sử dụng lại lệnh `sqlite3`:

   ```bash
   sqlite3 ten_file_moi.db
   ```

   Sau đó, bạn có thể sử dụng các câu lệnh như `SELECT` để kiểm tra dữ liệu.

### Ví dụ:

Giả sử bạn có một file `database.sql` với nội dung như sau:

```sql
CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  email TEXT NOT NULL
);

INSERT INTO users (name, email) VALUES ('Alice', 'alice@example.com');
INSERT INTO users (name, email) VALUES ('Bob', 'bob@example.com');
```

Bạn có thể tạo file `.db` từ file `.sql` này bằng lệnh sau:

```bash
sqlite3 my_database.db < database.sql
```

Sau khi thực hiện, file `my_database.db` sẽ được tạo ra và chứa các bảng cùng dữ liệu từ file SQL.

### Lưu ý:
- Nếu bạn sử dụng DBMS khác như MySQL hoặc PostgreSQL, quy trình sẽ khác và yêu cầu bạn nhập lệnh SQL vào một máy chủ cơ sở dữ liệu đã được cấu hình sẵn.
- SQLite rất thích hợp để làm việc với file `.db` đơn giản, vì toàn bộ cơ sở dữ liệu được lưu trong một file duy nhất, dễ dàng sao lưu và chia sẻ.

In [3]:
docs = []
loader = PyPDFLoader('../tmp/Phân tích cú pháp - Chương 4.pdf')
docs.extend(loader.load())

# Split documents and store in vector db
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectordb = FAISS.from_documents(splits, OpenAIEmbeddings())
vectordb.save_local("./vectordb")

# Define retriever
retriever = vectordb.as_retriever(
    search_type="mmr", search_kwargs={"k": 5}
)

c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [27]:
# Setup memory for contextual conversation
memory = ConversationBufferMemory(
    memory_key="chat_history", output_key="answer", return_messages=True
)
template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
llm=ChatOpenAI(
            model_name="gpt-4o-mini", temperature=0
        )

# Setup LLM and QA chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Chain
chain = prompt | llm


In [30]:
chain.invoke({'question': "phân tích cú pháp là gì", "chat_history": []})

AIMessage(content='Phân tích cú pháp là gì?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke({'question': "phân tích cú pháp là gì", "chat_history": []})

TypeError: expected string or buffer

In [3]:
from langchain_core.documents import Document

result = [(Document(metadata={'source': './tmp/Phân tích cú pháp - Chương 4.pdf', 'page': 0}, page_content='1Phân tích cú pháp\nLê Thanh Hương\nBộ môn Hệ thống Thông tin\nViện CNTT &TT –Trường ĐHBKHN\nEmail: huonglt@soict.hust.edu.vn'), 0.06666666666666667), (Document(metadata={'source': './tmp/Phân tích cú pháp - Chương 4.pdf', 'page': 4}, page_content='5Định nghĩa\n⚫Văn phạm ( grammar ) là dạng biểu diễn hình \nthức của các cấu trúc được chấp nhận trong \n1 ngôn ngữ\n⚫Thuật toán PTCP ( parsing algorithm ) là \nphương pháp xác định cấu trúc câu trên cơ \nsở ngữ pháp đã có.\n⚫Chương trình PTCP ( parser ) là chương trình \nxác định cấu trúc ngữ pháp của câu.'), 0.01639344262295082), (Document(metadata={'source': './tmp/Phân tích cú pháp - Chương 4.pdf', 'page': 17}, page_content='18Khó khăn với PTCP trên xuống\n⚫Các luật đệ qui trái\n⚫PTCP trên xuống rất bất lợi khi có nhiều luật có cùng vế trái\n⚫Nhiều thao tác thừa: triển khai tất cả các nút có thể phân tích trên \nxuống \n⚫PTCP trên xuống sẽ làm việc tốt khi có chiến lược điều khiển ngữ \npháp phù hợp\n⚫PTCP trên xuống không thể triển khai các ký hiệu tiền kết thúc \nthành các ký hiệu kết thúc. Trên thực tế, người ta thường sử dụng \nphương pháp dưới lên để làm việc này. \n⚫Lặp lại công việc: bất cứ chỗ nào có cấu trúc giống nhauS→NP X1 S→NP X2S→NP X600 S→VP Y1 ……'), 0.01639344262295082), (Document(metadata={'source': './tmp/Phân tích cú pháp - Chương 4.pdf', 'page': 27}, page_content='28Áp dụng thao tác ‘dán’\n12 3 45 6 7 8\n0DTNP\n1 NN\n2 VBD\n3 DT\n4 NN\n5 IN\n6 DT\n7 NN'), 0.01639344262295082), (Document(metadata={'source': './tmp/Phân tích cú pháp - Chương 4.pdf', 'page': 3}, page_content='44Các ứng dụng của PTCP\n▪Kiểm tra ngữ pháp (Microsoft)\n▪Trích rút thông tin (Hobbs 1996)\nKho văn bản\nNY TimesCSDL\ncâu truy vấn'), 0.01639344262295082)]

In [5]:
import os
for idx, doc in enumerate(result, 1):
    filename = os.path.basename(doc[0].metadata["source"])
    page_num = doc[0].metadata["page"]
    ref_title = (
        f":blue[Reference {idx}: *{filename} - page.{page_num}*]"
    )
    print(ref_title)

:blue[Reference 1: *Phân tích cú pháp - Chương 4.pdf - page.0*]
:blue[Reference 2: *Phân tích cú pháp - Chương 4.pdf - page.4*]
:blue[Reference 3: *Phân tích cú pháp - Chương 4.pdf - page.17*]
:blue[Reference 4: *Phân tích cú pháp - Chương 4.pdf - page.27*]
:blue[Reference 5: *Phân tích cú pháp - Chương 4.pdf - page.3*]
